In [345]:
import json, csv, copy

In [367]:
MAX_RULES = 4
ROWS_TO_PROCESS = 35000

In [368]:
#extract rules from a row
def get_rule(row,number=None):
    suffix = "_"+str(number) if number else ""
     
    rule = {}
    

   
    what = {}
    
    if len(row.get('zone'+suffix)):
        what['zone'] = row.get('zone'+suffix)
    
    if row.get('reason'+suffix):
        what['reason'] = row.get('reason'+suffix)
    
    if row.get('time_limit'+suffix):
        what['time_limit'] = int(row.get('time_limit'+suffix))
        
    if row.get('payment'+suffix):
        what['payment'] = row.get('payment'+suffix)
    
    if len(what):
        rule['what'] = what

    
    if row.get('priority'+suffix):
        rule['priority'] = int(row.get('priority'+suffix))
                

        
    who = {}
    
    if row.get('class'+suffix):
        who['class'] = row.get('class'+suffix)    
    
    if len(who):
        rule['who'] = who
        
    when = {}
     
    #Start with the timing:
    if row.get('days_of_week.days'+suffix):
        when['days_of_week'] = {
            'days' : [x.strip() for x in row.get('days_of_week.days'+suffix).split(",")]
        }
        
    if row.get('time_of_day.from'+suffix):
        when['time_of_day'] = {
            'from':row.get('time_of_day.from'+suffix),
            'to'  :row.get('time_of_day.to'+suffix) #Could add defaults here if necessary
        }
    
    if len(when):
        rule['when'] = [when]
    
    #Now check if there are other days or other times of day:
    for suffix2 in ['b','c','d','e','f','g','h','i']:
        new_when = {}
            
        if row.get('days_of_week.days'+suffix+suffix2):
            new_when['days_of_week'] = [x.strip() for x in row.get('days_of_week.days'+suffix+suffix2).split(",")]


        if row.get('time_of_day.from'+suffix+suffix2):
            new_when['time_of_day'] = {
                'from':row.get('time_of_day.from'+suffix+suffix2),
                'to'  :row.get('time_of_day.to'+suffix+suffix2) #Could add defaults here if necessary
            }

        if new_when:
            rule['when'].append(new_when)
        
#     if len(rule['when']) == 1:
#         rule['when'] = rule['when'][0]
        
        
    payment = {}
    
    if row.get('payment_min'+suffix):
        payment['min_rate'] = {
            'min_amount': float(row.get('payment_min'+suffix))
        }
        
        if len(row.get('payment_min_interval' + suffix)):
            payment['min_rate']['min_interval'] = int(row.get('payment_min_interval' + suffix))
        
    if row.get('payment_max'+suffix):
        payment['max_rate'] = {
            'max_amount': float(row.get('payment_max'+suffix))
        }     
       
        if len(row.get('payment_max_interval' + suffix)):
            payment['max_rate']['max_interval'] = int(row.get('payment_max_interval' + suffix))

        
    if row.get('method'+suffix):
        payment['method'] = row.get('method'+suffix)
        
    if row.get('payment_form'+suffix):
        payment['payment_form'] = row.get('payment_form'+suffix)
        
    if len(payment):
        rule['payment'] = payment
    
    return rule

In [372]:
#Runs everything

output_rows = []
with open('prepped_data.csv','r') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        line_count +=1;
        if line_count < ROWS_TO_PROCESS:

            
            #object level immutables
            obj = {
                'x':float(row['X']),
                'y':float(row['Y']),
                'shst_ref_id':row['ShStRefID'],      
                'shst_location':float(row['location_start']),
                'pt_relation':int(row['pt_relation']),
                'direction':row['direction'],
                'side_of_street':row['sideOfStreet'],
            }
            
            obj['rule'] = get_rule(row,"")
            
            if len(obj['rule']):
                output_rows.append(copy.deepcopy(obj))
            
            for i in range(2, MAX_RULES + 1):
                if row.get("zone_"+str(i)):
                    obj['rule'] = get_rule(row,i)
                    output_rows.append(copy.deepcopy(obj))


with open('output.json','w') as outFile:
    json.dump(output_rows,outFile,indent=2)
    


In [373]:
output_rows_geojson = []
for item in output_rows:
    output_rows_geojson.append({
        'type':"Feature",
        'geometry':{
            'type':"Point",
            'coordinates':[float(item["x"]),float(item["y"])]
        },
        'properties': item
        
    })


In [374]:
with open('output.geojson','w') as outFile:
    json.dump({'type':'FeatureCollection','features':output_rows_geojson},outFile,indent=2)